*******************************************************0. DATOS DEL DATAFRAME ******************************************************

*****1.VER DATOS

In [ ]:
df.info()

df.head()

df.shape[0]#Cuantas filas

df.shape[1]#Cuantas columnas

df.shape #Cuantas filas, column

********************************************************1. VALORES NULOS *****************************************************


*****1. VER Y ENCONTRAR NULOS


In [ ]:
df['Sex'].isnull().sum()  #De una columna en particular: Da un Nº 0 o 10 o 20

df['Embarked'].isnull().value_counts() # False	1307  True	2

na_fare = df[df['Fare'].isnull()] #Pasar LA (1) fila nula a una nueva

*****2. BORRAR NULOS


In [ ]:
#***** 2. BORRAR FILA CON " "
df.drop(df[df['workclass'] == ' '].index, inplace = True) #Borrar la Fila con " "
 
#***** 2. BORRAR FILA CON NULO
df.dropna(subset=['nombre_columna'], inplace=True)#BORRAR FILAS DE COLUMNA ESPECIFICA CON NULOS

cdf = df.dropna() #BORRAR TODAS LAS FILAS CON NULOS

#***** 2. BORRAR COLUMNA CON MUCHOS NULOS
df = df.drop(columns=['Cabin'])




*****3. ENCONTRAR MEDIA,MEDIANA ANTES DE REEMPLAZAR NULOS (PARA DATOS NUMERICOS)

#*****3.2. ENCONTRAR LA MEDIA COMO VALOR APARTE A PARTIR DE COLUMNAS IMPORTANTES
var1 = df['Fare'].loc[ (df['Ticket'] == na_fare['Ticket'].values[0]) &
                            (df['Pclass'] == na_fare['Pclass'].values[0]) &
                            (df['Embarked'] == na_fare['Embarked'].values[0]) ].mean()

#*****5.1. ENCONTRAR LA MEDIANA COMO VALOR APARTE
guess_Fare = df['Fare'].loc[ (df['Ticket'] == na_fare['Ticket'].values[0]) &
                            (df['Pclass'] == na_fare['Pclass'].values[0]) &
                            (df['Embarked'] == na_fare['Embarked'].values[0])].median()


*****4. REEMPLAZAR NULOS CON: <br>
NÚMEROS, MEDIA, MEDIANA, MODA, AGRUPACIONES A PARTIR DE OTRAS O RANDOMREGRESOR (CON PREDICCIONES)

In [ ]:
#*****1. REEMPLAZAR NULOS CON NÚMEROS
df['hours-per-week'].fillna(0, inplace=True) #Con 0s 

#*****2. REEMPLAZAR NULOS CON MEDIA DE LA COLUMNA DIRECTAMENTE, ES MAS GENERAL
df['hours-per-week'].fillna(df['hours-per-week'].mean(), inplace=True)

#*****3. REEMPLAZAR NULOS CON MEDIANA 
df['Fare'].fillna(guess_Fare, inplace=True) #del ejercicio 5.1 A PARTIR DE LA EQUIVALENCIA DE OTRAS (media específica)

#*****4. REEMPLAZAR NULOS CON MEDIANA DE LA COLUMNA DIRECTAMENTE, ES MAS GENERAL
df['hours-per-week'].fillna(df['hours-per-week'].median(), inplace=True) 

#*****5. REEMPLAZAR NULOS CON MODA (SOLO PARA DATOS CATEGORICOS)
df['Embarked'].fillna(df['Embarked'].mode()[0], inplace=True)#moda

#*****6. REEMPLAZAR NULOS CON AGRUPACIONES CON COLUMNAS CO-RELACIONADAS
group = df[['education','education-num']].groupby('education').mean().reset_index() #Se crea columna group

#df[['education', 'education-num']]: Selecciona las columnas education y education-num
#.groupby('education'): Agrupa los datos por la columna education, lo que significa que los datos se agruparán según cada nivel educativo.
#.mean(): Calcula el promedio de la columna education-num para cada grupo de education.
#.reset_index(): Convierte el resultado de la operación de agrupación en un nuevo DataFrame

In [ ]:
#*****7. REEMPLAZAR NULOS CON PREDICCIONES: RANDOMFORESTREGRESSOR
from sklearn.ensemble import RandomForestRegressor

# ¿Qué variables nos interesan para tener en cuenta?
df_sub = df[['Age','Title','Fare','SibSp']]

# Para poder entrenar el modelo y comprobar que funciona adecuadamente para aplicarlo a los datos desconocidos debemos:

# 1. eliminar aquellas filas nulas de los datos de entrenamiento
X_train  = df_sub.dropna().drop('Age', axis=1)
y_train  = df['Age'].dropna()

# 2. guardar en test las filas nulas #SON LAS QUE QUEREMOS PREDECIR
X_test = df_sub.loc[np.isnan(df.Age)].drop('Age', axis=1)

# 3.Aplicamos el algortimo Con los datos de entrenamiento
regressor = RandomForestRegressor(n_estimators = 300)
regressor.fit(X_train, y_train)

# 4.Predecimos los valores que nos faltan
y_pred = np.round(regressor.predict(X_test),1) #CON X_tEST
y_pred

#5.Sustituimos estos valores en el dataset
df.Age.loc[df.Age.isnull()] = y_pred

#6.Sustituimos estos valores en el dataset
df.Age.isnull().sum(axis=0)

EL PROCESO ANTERIOR SE HACE INDEPENDIENTE DE LA CATEGORÍA DEL DATO, AHORA REALIZAREMOS ACCIONES, DEPENDIENDO DEL TIPO DE DATO (CATEGORICO O NUMERICO)

****************************************************** 3. DATOS CATEGORICOS *******************************************************


#*****3.1. VER CATEGORIAS

In [ ]:
#*****3. VER TIPOS
df['Sex'].unique() #array(['male', 'female'])

#*****3. VER CUANTOS DATOS TIENE CADA CATEGORIA
df['Ticket'].value_counts()

#*****3.2. TRANSFORMAR A VARIABLES CATEGORICAS MÁS SIMPLES

#*****1. MANTENER SÓLO LA INICIAL
df['Ticket'] = df['Ticket'].map(lambda k: k[0]) #"A/5 21171" Transforma esa variable en "A", solo deja la Inicial

#*****2. SUSTITUIR ALGUNAS VARIABLES POR UN VALOR ESPECIFICO
df['Ticket'] = df['Ticket'].replace(['A','W','F','L','5','6','7','8','9'], '4') #Sustituye todas esas por 4

#*****3. DARLES CLASES ESPECÍFICAS , LUEGO CADA CLASE SE DEBE TRANSFORMAR
def get_titles(df):
    title_dictionary = {
        "Capt": "Officer",
        "Col": "Officer",
        "Major": "Officer",
        "Jonkheer": "Royalty",
        "Don": "Royalty",
        "Dona": "Royalty",
        "Sir": "Royalty",
        "Dr": "Officer",
        "Rev": "Officer",
        "the Countess": "Royalty",
        "Mme": "Mrs",
        "Mlle": "Miss",
        "Ms": "Mrs",
        "Mr": "Mr",
        "Mrs": "Mrs",
        "Miss": "Miss",
        "Master": "Master",
        "Lady": "Royalty"
    }
    print(df['Name'])

    def process_name(x):
        # print(x)
        return x.split(',')[1].split('.')[0].strip()

    df['Title'] = df['Name'].map(process_name)
    df['Title'] = df.Title.map(title_dictionary)
    print(df['Name'][df['Title'].isna()])

    return df

#*****3.3. TRANSFORMAR A NÚMEROS

In [ ]:
#*****1. SUSTITUCIÓN: MAP

df['Sex'] = df['Sex'].map({'male': 0, 'female': 1}) #Ej.1
df['Embarked'] = df['Embarked'].map({'S': 2, 'C': 0, 'Q': 1}) #Ej.2

df = get_titles(df) #Ej.3 del ejemplo anterior
df['Title'] = df['Title'].map({'Officer': 0, 'Royalty': 1, 'Mrs': 2, 'Miss': 3, 'Master': 4, 'Mr': 5}).astype(int)

#*****2. ENCODE_LABEL

def encode_label(df):
    return df.astype('category').cat.codes

df['Ticket'] = encode_label(df['Ticket'])

#*****3.LABEL ENCODER
from sklearn import preprocessing
encoder = preprocessing.LabelEncoder()

encoded = encoder.fit_transform(df['workclass'])
df['workclass'] = encoded.astype('int')


****************************************************** 4. DATOS NUMERICOS *******************************************************


#*****4.1. TRANSFORMAR A MENOS VARIABLES NUMÉRICAS

In [ ]:
#*****1. BINS:
queda Fare-bin como la misma columna solo que con contenedores de datos más reducidos
df['Fare-bin'] = pd.qcut(df.Fare,5,labels=[1,2,3,4,5]).astype(int)


#*****1.2 BINS:
Transformar un rango de edades continuas en categorías como "0-18", "19-35", "36-50", "51-70", "71+".
df['Age-bin'] = pd.cut(df['Age'], bins=[0, 18, 35, 50, 70, 100], labels=['0-18', '19-35', '36-50', '51-70', '71+'])

#*****1.3 BINS:
df['Income-bin'] = pd.cut(df['Income'], bins=[0, 30000, 60000, 100000], labels=['Low', 'Medium', 'High'])


In [ ]:
#*****2. AGRUPACIONES:
Se llama discretización: simplifica un rango continuo en categorías numeradas en UNA sola columna.

df['Age-bin'] = df['Age']
df.loc[df['Age-bin'] <= 16, 'Age-bin'] = 0
df.loc[(df['Age-bin'] > 16) & (df['Age-bin'] <= 32), 'Age-bin'] = 1
df.loc[(df['Age-bin'] > 32), 'Age-bin'] = 2

In [ ]:
#****3. "DUMMIES": creación de variables indicadoras o encoding con variables dummies 
Son variables indicadoras: genera una columna separada para cada categoría, asignando valores binarios (1 o 0).

df['FamilySize'] = df['Parch'] + df['SibSp'] + 1
df['Singleton'] = df['FamilySize'].map(lambda s: 1 if s == 1 else 0)
df['SmallFamily'] = df['FamilySize'].map(lambda s: 1 if 2 <= s <= 3 else 0)
df['MediumFamily'] = df['FamilySize'].map(lambda s: 1 if s == 4 else 0)
df['LargeFamily'] = df['FamilySize'].map(lambda s: 1 if 5 <= s else 0)